In [1]:
import pandas as pd
import locale
import glob
import os.path
import requests
import tarfile


# Convert text to lower-case and strip punctuation/symbols from words
def normalize_text(text):
    norm_text = text.lower()

    # Replace breaks with spaces
    norm_text = norm_text.replace('<br />', ' ')

    # Pad punctuation with spaces on both sides
    for char in ['.', '"', ',', '(', ')', '!', '?', ';', ':']:
        norm_text = norm_text.replace(char, ' ' + char + ' ')

    return norm_text


In [2]:
fields = ['text', 'stars']

df = pd.read_csv('/scratch/qx344/yelp/yelp_academic_dataset_review.csv', skipinitialspace=True, usecols=fields,nrows = 100000)
# See the keys
print df.keys(),len(df.index)

Index([u'text', u'stars'], dtype='object') 100000


In [3]:
import gensim
from gensim.models.doc2vec import TaggedDocument
from collections import namedtuple
import io

SentimentDocument = namedtuple('SentimentDocument', 'words tags split sentiment')

alldocs = []  # will hold all docs in original order

#for line_no, line in enumerate(alldata):
for line_no, row in df.iterrows():
    line = row['text']
    tokens = gensim.utils.to_unicode(line).split()
    #words = tokens[1:]
    words = tokens
    tags = [line_no] # `tags = [tokens[0]]` would also work at extra memory cost
    split = ['train','train','train','test'][line_no//25000]  # 75k train, 25k test
    #sentiment = [1.0, 0.0, 1.0, 0.0, None, None, None, None][line_no//12500] # [12.5K pos, 12.5K neg]*2 then unknown
    sentiment = row['stars']*1.0
    #sentiment = 0 if row['stars']<3 else 1
    alldocs.append(SentimentDocument(words, tags, split, sentiment))

train_docs = [doc for doc in alldocs if doc.split == 'train']
test_docs = [doc for doc in alldocs if doc.split == 'test']
doc_list = alldocs[:]  # for reshuffling per pass

print('%d docs: %d train-sentiment, %d test-sentiment' % (len(doc_list), len(train_docs), len(test_docs)))

100000 docs: 75000 train-sentiment, 25000 test-sentiment


In [10]:
from gensim.models import Doc2Vec
import gensim.models.doc2vec
from collections import OrderedDict
import multiprocessing

cores = multiprocessing.cpu_count()
assert gensim.models.doc2vec.FAST_VERSION > -1, "this will be painfully slow otherwise"

simple_models = [
    # PV-DM w/concatenation - window=5 (both sides) approximates paper's 10-word total window size
    Doc2Vec(dm=1, dm_concat=1, size=80, window=5, negative=5, hs=0, min_count=2, workers=cores),
    # PV-DBOW 
    Doc2Vec(dm=0, size=80, negative=5, hs=0, min_count=2, workers=cores),
    # PV-DM w/average
    Doc2Vec(dm=1, dm_mean=1, size=80, window=10, negative=5, hs=0, min_count=2, workers=cores),
]

# speed setup by sharing results of 1st model's vocabulary scan
simple_models[0].build_vocab(alldocs)  # PV-DM/concat requires one special NULL word so it serves as template
print(simple_models[0])
for model in simple_models[1:]:
    model.reset_from(simple_models[0])
    print(model)

models_by_name = OrderedDict((str(model), model) for model in simple_models)

Doc2Vec(dm/c,d80,n5,w5,mc2,t20)
Doc2Vec(dbow,d80,n5,mc2,t20)
Doc2Vec(dm/m,d80,n5,w10,mc2,t20)


In [11]:
from gensim.test.test_doc2vec import ConcatenatedDoc2Vec
models_by_name['dbow+dmm'] = ConcatenatedDoc2Vec([simple_models[1], simple_models[2]])
models_by_name['dbow+dmc'] = ConcatenatedDoc2Vec([simple_models[1], simple_models[0]])

In [12]:
import numpy as np
import statsmodels.api as sm
from random import sample

# for timing
from contextlib import contextmanager
from timeit import default_timer
import time 

@contextmanager
def elapsed_timer():
    start = default_timer()
    elapser = lambda: default_timer() - start
    yield lambda: elapser()
    end = default_timer()
    elapser = lambda: end-start
    
    
    
    
from sklearn import linear_model


def error_rate_for_model(test_model, train_set, test_set, infer=False, infer_steps=3, infer_alpha=0.1, infer_subsample=0.1):
    """Report error rate on test_doc sentiments, using supplied model and train_docs"""

    train_targets, train_regressors = zip(*[(doc.sentiment, test_model.docvecs[doc.tags[0]]) for doc in train_set])
    #train_regressors = sm.add_constant(train_regressors)
    logit = linear_model.LogisticRegression(penalty='l2', dual=False, tol=0.0001, C=1.0, fit_intercept=True, 
                                        intercept_scaling=1, class_weight=None, random_state=None, solver='lbfgs', max_iter=300, 
                                        multi_class='multinomial', verbose=0, warm_start=False, n_jobs=1)
    predictor = logit.fit(train_regressors,train_targets)

    test_targets, test_regressors = zip(*[(doc.sentiment, test_model.docvecs[doc.tags[0]]) for doc in test_set])
    
    corrects = predictor.score(test_regressors,test_targets)
    error_rate= 1 - corrects
    errors = len(test_targets)*error_rate
    return (error_rate, errors, len(test_targets), predictor)

In [13]:
from collections import defaultdict
best_error = defaultdict(lambda :1.0)  # to selectively-print only best errors achieved

In [ ]:
from random import shuffle
import datetime

alpha, min_alpha, passes = (0.025, 0.001, 20)
alpha_delta = (alpha - min_alpha) / passes

print("START %s" % datetime.datetime.now())

for epoch in range(passes):
    shuffle(doc_list)  # shuffling gets best results
    
    for name, train_model in models_by_name.items():
        # train
        duration = 'na'
        train_model.alpha, train_model.min_alpha = alpha, alpha
        with elapsed_timer() as elapsed:
            train_model.train(doc_list)
            duration = '%.1f' % elapsed()
            
        # evaluate
        eval_duration = ''
        with elapsed_timer() as eval_elapsed:
            err, err_count, test_count, predictor = error_rate_for_model(train_model, train_docs, test_docs)
        eval_duration = '%.1f' % eval_elapsed()
        best_indicator = ' '
        if err <= best_error[name]:
            best_error[name] = err
            best_indicator = '*' 
        print("%s%f : %i passes : %s %ss %ss" % (best_indicator, err, epoch + 1, name, duration, eval_duration))

#         if ((epoch + 1) % 5) == 0 or epoch == 0:
#             eval_duration = ''
#             with elapsed_timer() as eval_elapsed:
#                 infer_err, err_count, test_count, predictor = error_rate_for_model(train_model, train_docs, test_docs, infer=True)
#             eval_duration = '%.1f' % eval_elapsed()
#             best_indicator = ' '
#             if infer_err < best_error[name + '_inferred']:
#                 best_error[name + '_inferred'] = infer_err
#                 best_indicator = '*'
#             print("%s%f : %i passes : %s %ss %ss" % (best_indicator, infer_err, epoch + 1, name + '_inferred', duration, eval_duration))

    print('completed pass %i at alpha %f' % (epoch + 1, alpha))
    alpha -= alpha_delta
    
print("END %s" % str(datetime.datetime.now()))